In [ ]:
!pip install num2words
%pip install -q ipdb
%env PYTHONBREAKPOINT=IPython.core.debugger.set_trace
%pdb on

In [ ]:
!pip3 install lerobot==0.3.3

In [3]:
import torch
from lerobot.policies.smolvla.modeling_smolvla import SmolVLAPolicy
from lerobot.policies.smolvla.configuration_smolvla import SmolVLAConfig
from lerobot.configs.policies import PreTrainedConfig

from lerobot.datasets.lerobot_dataset import LeRobotDatasetMetadata

device = "cuda" if torch.cuda.is_available() else "cpu"
# config = SmolVLAConfig()
# config = SmolVLAConfig.from_pretrained("lerobot/smolvla_base")
config = PreTrainedConfig.from_pretrained("lerobot/smolvla_base")
ds_meta = LeRobotDatasetMetadata("lerobot/svla_so101_pickplace")
policy = SmolVLAPolicy(config, dataset_stats=ds_meta.stats).to(device).eval()
# policy = SmolVLAPolicy("lerobot/smolvla_base").to(device).eval()
# policy = SmolVLAPolicy.from_pretrained("lerobot/smolvla_base").to(device).eval()

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


config.json: 0.00B [00:00, ?B/s]

Fetching 4 files:   0%|          | 0/4 [00:00<?, ?it/s]

info.json: 0.00B [00:00, ?B/s]

episodes_stats.jsonl: 0.00B [00:00, ?B/s]

...

model.safetensors:   0%|          | 0.00/2.03G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/136 [00:00<?, ?B/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Fetching 2 files:   0%|          | 0/2 [00:00<?, ?it/s]

Reducing the number of VLM layers to 16 ...


In [4]:
from lerobot.envs.utils import preprocess_observation
import numpy as np
import ipdb

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

H, W = 256, 256

observation = {
    "pixels":    np.random.randint(0, 256, (H, W, 3), dtype=np.uint8),
    # "agent_pos": np.zeros(state_dim, dtype=np.float32),
    "agent_pos": np.zeros(6, dtype=np.float32),
}

observation = preprocess_observation(observation)
observation = {k: v.to(device, non_blocking=(device.type=="cuda")) for k, v in observation.items()}
observation["task"] = ["stack the blocks"]

In [5]:
policy.model.vlm_with_expert.expert_hidden_size

720

In [ ]:
ipdb.set_trace()
with torch.inference_mode():
    action = policy.select_action(observation)
print(action)

--Return--
None
> /tmp/ipython-input-151337218.py(1)<cell line: 0>()
----> 1 ipdb.set_trace()
      2 with torch.inference_mode():
      3     action = policy.select_action(observation)

ipdb> b /usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/modeling_smolvla.py:929
Breakpoint 1 at /usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/modeling_smolvla.py:929
ipdb> r
    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

    [... skipped 1 hidden frame]

Internal StopIteration: False
> /usr/local/lib/python3.12/dist-packages/IPython/core/interactiveshell.py(3473)run_ast_nodes()
   3472                         asy = compare(code)
-> 3473                     if (await self.run_code(code, result,  async_=asy)):
   3474                         return True

ipdb> r
> /usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/modeling_smolvla.py(929)denoise_step()
    928 
1-> 929         outputs_embeds, _ = self

b /usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/modeling_smolvla.py:893

b /usr/local/lib/python3.12/dist-packages/lerobot/policies/smolvla/modeling_smolvla.py:807